In [54]:
import simpy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import random

RANDOM_SEED = 42
NUM_SERVER = 2 # number of servers in the simulation
SERVICETIME = 1.5 # average? minutes to complete a task
T_ARRIVAL = 7 # arrival of a new customer
SIM_TIME = 50 # Simulation length in minutes
rho = 0.5
mu = 2
lambd = rho * mu

class Server(object):
    
    def __init__(self, env, num_server, servicetime):
        
        self.env = env
        self.server = sp.Resource(env, num_server)
        self.servicetime = servicetime
        
    def service(self, customer):
        # need to add stochasticity to this
        
        yield self.env.timeout(randomSample(SERVICETIME))
        
def randomSample(rate):
    
    return np.random.exponential(rate)
        
def client(env, name, sv, waiting):
    
    arrival = env.now
    print("%s arrives at %.2f." %(name, arrival))
    
    with sv.server.request() as request:
        
        yield request
        
        service = env.now
        print("%s is processed at %.2f." % (name, service))
        
        if arrival != service:
            waiting[1] += 1
            waiting[2] += service - arrival
        
        yield env.process(sv.service(name))
        
        print("%s is finished at %.2f." %(name, env.now))
        
        
def setup(env, num_servers, servicetime, t_arrival, waiting):
    
    server = Server(env, num_servers, servicetime)
    
    # create 4 customers
    for i in range(4):
        waiting[0] += 1
        env.process(client(env, 'Client %d' % i, server, waiting))
        
    # make more customers while the simulation is running
    while waiting[0]<40:
        yield env.timeout(randomSample(1/lambd))
        waiting[0] += 1 
        
        env.process(client(env, 'Client %d' % waiting[0], server, waiting))
        print(waiting)#[1],waiting[2])
        

def runSimulation(serverNumber, waiting):
    
    print('Server is up')
    random.seed(RANDOM_SEED)

    # create the enviroment and start the setup process

    env = sp.Environment()
    env.process(setup(env, serverNumber, SERVICETIME, T_ARRIVAL, waiting))

    env.run()
    
    

In [55]:
waiting = [0,0,0]

NUM_SERVER = 2

runSimulation(NUM_SERVER, waiting)

print("The total waiting time was %.2f" %(waiting[2]))
print("There were %i clients, %i of who(m?) had to wait." %(waiting[0],waiting[1]))
print("The average waiting time was %.2f" %(waiting[2]/waiting[0]))

#This isnt right and needs to be checked

Server is up
Client 0 arrives at 0.00.
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 0 is processed at 0.00.
Client 1 is processed at 0.00.
Client 0 is finished at 0.29.
Client 2 is processed at 0.29.
Client 2 is finished at 0.41.
Client 3 is processed at 0.41.
[5, 2, 0.6954322797223988]
Client 5 arrives at 0.48.
[6, 2, 0.6954322797223988]
Client 6 arrives at 1.23.
Client 1 is finished at 1.86.
Client 5 is processed at 1.86.
Client 3 is finished at 1.90.
Client 6 is processed at 1.90.
[7, 4, 2.7476537005495523]
Client 7 arrives at 2.05.
Client 6 is finished at 2.71.
Client 7 is processed at 2.71.
Client 5 is finished at 2.72.
[8, 5, 3.412212879755644]
Client 8 arrives at 3.73.
Client 8 is processed at 3.73.
Client 7 is finished at 4.44.
Client 8 is finished at 5.92.
[9, 5, 3.412212879755644]
Client 9 arrives at 8.09.
Client 9 is processed at 8.09.
[10, 5, 3.412212879755644]
Client 10 arrives at 8.27.
Client 10 is processed at 8.27.
Client 9 is fin

the link https://simpy.readthedocs.io/en/latest/examples/carwash.html

In [57]:
waiting = [0,0,0]

NUM_SERVER = 1

runSimulation(NUM_SERVER, waiting)

print("The total waiting time was %.2f" %(waiting[2]))
print("There were %i clients, %i of who(m?) had to wait." %(waiting[0],waiting[1]))
print("The average waiting time was %.2f" %(waiting[2]/waiting[0]))

Server is up
Client 0 arrives at 0.00.
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 0 is processed at 0.00.
[5, 0, 0]
Client 5 arrives at 0.93.
[6, 0, 0]
Client 6 arrives at 1.88.
Client 0 is finished at 2.52.
Client 1 is processed at 2.52.
Client 1 is finished at 2.67.
Client 2 is processed at 2.67.
[7, 2, 5.1837468834179585]
Client 7 arrives at 2.82.
Client 2 is finished at 2.90.
Client 3 is processed at 2.90.
Client 3 is finished at 3.10.
Client 5 is processed at 3.10.
[8, 4, 10.258011599974653]
Client 8 arrives at 3.30.
[9, 4, 10.258011599974653]
Client 9 arrives at 3.45.
Client 5 is finished at 3.79.
Client 6 is processed at 3.79.
[10, 5, 12.171815998442732]
Client 10 arrives at 4.33.
Client 6 is finished at 4.36.
Client 7 is processed at 4.36.
[11, 6, 13.705548816903711]
Client 11 arrives at 5.69.
Client 7 is finished at 6.31.
Client 8 is processed at 6.31.
[12, 7, 16.71540700495388]
Client 12 arrives at 8.15.
[13, 7, 16.71540700495388]
Cli